#### Пример 13.1.Частичная реализация протокола последовательности – метода __getitem__

In [2]:
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]

v = Vowels()
v[0]

'A'

In [4]:
v[-1]

'U'

In [3]:
for c in v: 
    print(c)

A
E
I
O
U


In [5]:
'E' in v

True

In [6]:
'Z' in v

False

#### Пример 13.2. Колода как последовательность карт (тот же код, что в примере 1.1)

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

#### Партизанское латание как средство реализации протокола во время выполнения

In [8]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[6, 4, 7, 1, 8, 3, 5, 0, 2, 9]

#### Пример 13.3. random.shuffle не может работать с объектом FrenchDeck

In [9]:
from random import shuffle
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

#### Пример 13.4. Партизанское латание класса FrenchDeck с целью сделать его изменяемым и совместимым с функцией random.shuffle (продолжение примера 13.3)

In [10]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='K', suit='diamonds'),
 Card(rank='7', suit='hearts'),
 Card(rank='2', suit='spades'),
 Card(rank='Q', suit='clubs'),
 Card(rank='A', suit='diamonds')]

#### Пример 13.5. Применение утиной типизации для обработки строки или итерируемого объекта строк

In [11]:
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    pass
field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise ValueError('field_names must all be valid identifiers')

NameError: name 'field_names' is not defined

#### Пример 13.6. frenchdeck2.py: FrenchDeck2, подкласс collections.MutableSequence